In [2]:
import pandas as pd
matches = pd.read_csv("matches_long.csv", index_col = 0)
matches.shape
# total_game_count = 38*20*2 # our numbers align, good!
#matches["team"].value_counts() #this checks games per team


(1520, 33)

In [3]:
# cleaning the data, ensuring only numerical data

matches.dtypes #check data types of each col

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf                int64
ga                int64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
totdist         float64
prgdist         float64
err             float64
touches         float64
def 3rd         float64
att 3rd         float64
season            int64
team             object
dtype: object

In [4]:
matches["date"] = pd.to_datetime(matches["date"]) # changing the type of the "date" col
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                       int64
ga                       int64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                       int64
pkatt                    int64
totdist                float64
prgdist                float64
err                    float64
touches                float64
def 3rd                float64
att 3rd                float64
season                   int64
team    

In [5]:
#changing to numerical categories, creating new columns w these numericla versions
matches["venue_code"] = matches["venue"].astype("category").cat.codes #convert venue to numerical (0 for away or 1 for home)
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int") #change time to just the hour
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"]  = (matches["result"]=="W").astype("int")


matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,touches,def 3rd,att 3rd,season,team,venue_code,opp_code,hour,day_code,target
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,835.0,267.0,158.0,2024,Manchester City,0,5,20,4,1
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,788.0,212.0,174.0,2024,Manchester City,1,16,20,5,1
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,893.0,153.0,304.0,2024,Manchester City,0,18,14,6,1
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,838.0,222.0,138.0,2024,Manchester City,1,9,15,5,1
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,837.0,148.0,311.0,2024,Manchester City,0,21,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,485.0,236.0,68.0,2023,Southampton,0,16,14,6,0
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,688.0,159.0,166.0,2023,Southampton,0,17,20,0,0
44,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Fulham,...,475.0,157.0,100.0,2023,Southampton,1,9,15,5,0
45,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1,3,Brighton,...,499.0,268.0,76.0,2023,Southampton,0,4,14,6,0


In [6]:
# Random Forect Classifier, good for picking up non-linearities!
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 200, min_samples_split = 4, min_samples_leaf= 2, max_depth= None, random_state = 1) # numebr of estimators, and the higher the min_samples_split number is, the lower accuracy but less likely to overfit
train =  matches[matches["date"] < '2024-01-01'] # time series, so train has to be older data!
test = matches[matches["date"] > '2024-01-01']

predictors = ["venue_code", "opp_code", "hour", "day_code"] #attributes being used
rf.fit(train[predictors], train["target"]) # use the predictors to predict the target
preds = rf.predict(test[predictors])

from sklearn.metrics import accuracy_score
acc = accuracy_score(test["target"], preds)
acc # was 64%

0.6366120218579235

In [7]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"]) # creating a prediction/actual table

predicted,0,1
actual,,
0,176,53
1,80,57


In [8]:
from sklearn.metrics import precision_score
precision_score(test["target"], preds) # was 52% (want to about 75%)

0.5181818181818182

In [67]:
#improve using rolling avgs
def rolling_avg(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean() # ensures no future games are used, only previous 3 weeks
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

#cols = ["gf", "ga","xg", "xga","poss","sh", "sot", "dist", "fk", "pk", "pkatt", "totdist", "prgdist", "err", "touches", "def 3rd", "att 3rd"] # add in all the detailed attributes
cols = ["gf", "ga", "poss","sh", "sot", "dist", "fk", "pk", "pkatt", "prgdist", "err", "touches", "def 3rd", "att 3rd"] # add in all the detailed attributes

new_cols = [f"{c}_rolling" for c in cols] #create new cols of these rolling avgs

In [68]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_avg(x, cols, new_cols)) # apply rolling averages to every team
matches_rolling = matches_rolling.droplevel('team') # remove team on the left sife
matches_rolling.index = range(len(matches_rolling)) # fix so index goes from 0 - len
#matches_rolling

In [18]:
# create make pred function
def make_pred(data, predictors):
  train =  data[data["date"] < '2024-01-01'] # time series, so train has to be older data!
  test = data[data["date"] > '2024-01-01']

  rf.fit(train[predictors], train["target"]) # use the predictors to predict the target
  preds = rf.predict(test[predictors])

  combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index = test.index)
  precision = precision_score(test["target"], preds)
  return combined, precision

In [69]:
combined, precision = make_pred(matches_rolling, predictors + new_cols)
precision # 60.7%

0.6071428571428571

In [70]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
combined

,actual,predicted,date,team,opponent,result
55,1,1,2024-01-20,Arsenal,Crystal Palace,W
56,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
57,1,1,2024-02-04,Arsenal,Liverpool,W
58,1,0,2024-02-11,Arsenal,West Ham,W
59,1,0,2024-02-17,Arsenal,Burnley,W
...,...,...,...,...,...,...
1446,0,0,2024-04-24,Wolverhampton Wanderers,Bournemouth,L
1447,1,0,2024-04-27,Wolverhampton Wanderers,Luton Town,W
1448,0,0,2024-05-04,Wolverhampton Wanderers,Manchester City,L
1449,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L


In [72]:
#ensuring that both fixtures (as 2 teams play the same game, thus both must be correct) match up, this will give us our correct precision
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"}
mapping = MissingDict(**map_values)

combined["new_team"] = combined["team"].map(mapping)
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts() # 71%

actual_x
1    59
0    24
Name: count, dtype: int64

In [73]:
59/(59+24)

0.7108433734939759